In [7]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRConfig
from Utilities.ADTCR import ADTCRModel
from Utilities.UCRParser import read_dataset

In [8]:
testing_dataset = "Symbols"
train_ds, test_ds = read_dataset(testing_dataset)

config = DTCRConfig()
config.checkpoint_path = "ADTCR_Checkpoints"
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.model_name = testing_dataset
config.batch_size = len(train_ds)

config.learning_rate = 8e-5
config.checkpoint_interval = 50
config.hidden_size = [50, 30, 30]
config.dilations = [1, 4, 16]
config.coefficient_lambda = 0.1

train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_ds, batch_size=len(test_ds), shuffle=True)

Loading the Symbols dataset...
The dataset Symbols was loaded.


In [9]:
dtcr_model = ADTCRModel(config)
dtcr_model

ADTCRModel(
  (encoder): AttentionEncoder(
    (input_embedding): Linear(in_features=1, out_features=220, bias=True)
    (positional_embedding): Embedding(398, 220)
    (layers): ModuleList(
      (0): AttentionEncoderLayer(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=220, out_features=220, bias=True)
        )
        (first_norm): LayerNorm((220,), eps=1e-05, elementwise_affine=True)
        (ff): Sequential(
          (0): Linear(in_features=220, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=220, bias=True)
        )
        (second_norm): LayerNorm((220,), eps=1e-05, elementwise_affine=True)
      )
      (1): AttentionEncoderLayer(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=220, out_features=220, bias=True)
        )
        (first_norm): LayerNorm((220,), eps=1e-05, elementwise_affine=True)
        (ff): Sequ

In [10]:
# if you want to continue the training load the model here from the checkpoint
# dtcr_model = torch.load("Checkpoints/ECGFiveDays_900")
recons_criterion = config.decoding_criterion()
classify_criterion = config.classifier_criterion()
optimizer = config.optimizer(dtcr_model.parameters(),
                             eps=config.learning_rate)

In [11]:
epochs = 1500
for epoch in range(dtcr_model._training_iteration, epochs):
    print("Starting epoch {}:".format(epoch + 1))
    dtcr_model.train_step(train_dl, test_dl, recons_criterion, classify_criterion,
                          optimizer)

SyntaxError: invalid syntax (Temp/ipykernel_73248/408261322.py, line 1)